# Disclaimer
In the following, I have collected information from several sources. These are spread around in this notebook. The resulting text written here is thus a collection of what these authors have written in and between the lines. The misprints and imprecisions are mine though. Please do follow the links if what is written here doesn't make sense.  

# Seq2Seq model - Learn to Execute

**Source:** This is a notebook version of https://machinelearningmastery.com/learn-add-numbers-seq2seq-recurrent-neural-networks/ with some minor modifications. I'm not including the part of "the beginner's mistake" from this link, so you should take a look at that too.

If you want to go deeper, you can look at https://machinelearningmastery.com/lstms-with-python/ , chapter 9.

The [Encoder-Decoder LSTM](https://machinelearningmastery.com/encoder-decoder-long-short-term-memory-networks/) was developed for natural language processing problems where it demonstrated state-of-the-art performance, specifically in the area of text translation called statistical machine translation.

The list below highlights some interesting applications of the Encoder-Decoder LSTM architecture:

- Machine Translation, e.g. English to French translation of phrases.
- Learning to Execute, e.g. calculate the outcome of small programs.
- Image Captioning, e.g. generating a text description for images.
- Conversational Modeling, e.g. generating answers to textual questions.
- Movement Classification, e.g. generating a sequence of commands from a sequence of gestures.


## Architecture of encoder-decoder LSTM

The model is comprised of two parts: the _encoder_ and the _decoder_. First, the input sequence is
shown to the network one encoded character at a time. We need an encoding level to learn the
relationship between the steps in the input sequence and develop an internal representation of
these relationships.
One or more LSTM layers can be used to implement the encoder model. In the example we will use only one. The output of this
model is a fixed-size vector that represents the internal representation of the input sequence.
The number of memory cells in this layer defines the length of this fixed-sized vector.

**Side note:** This fixed-size vector seems to cause performance problems when working on long input or output sequences. To fix this, one can use [Attention](https://machinelearningmastery.com/implementation-patterns-encoder-decoder-rnn-architecture-attention/) which is not covered in this notebook. But here is an [example](https://machinelearningmastery.com/encoder-decoder-attention-sequence-to-sequence-prediction-keras/) to get you started.

The decoder must transform the learned internal representation of the input sequence into
the correct output sequence. One or more LSTM layers can also be used to implement the
decoder model. In the example, we will use only one. This model reads from the fixed sized output from the encoder model. Then we use a Dense layer as the output for the network. The same weights can
be used to output each time step in the output sequence by wrapping the Dense layer in a
[TimeDistributed wrapper](https://machinelearningmastery.com/timedistributed-layer-for-long-short-term-memory-networks-in-python/). You can also learn more about this in the keras documentation for [TimeDistributed](https://keras.io/layers/wrappers/#timedistributed).

The output from the encoder is a 2D vector. The input of the decoder is a 3D vector. To make them fit, we use an adaptor: the [RepeatVector](https://keras.io/layers/core/#repeatvector).

```python
model = Sequential()
model.add(LSTM(..., input_shape=(...)))  # encoder, output 2D
model.add(RepeatVector(...))  # adaptor, map 2D to 3D by repeating the input. 
model.add(LSTM(..., return_sequences=True))  # decoder, input 3D
model.add(TimeDistributed(Dense(...)))  # we reuse the same output layer for each element in the output sequence. 
                                     
```

## Addition Prediction Problem
We are going to work on an example of _Learning to Execute_, by calculating the sum output of two input numbers. This problem was used by Wojciech Zaremba and Ilya Sutskever in their 2014 paper, where this architecture was demostrated learning to calculate the output of small programs.

Example:
```python
Input: ['1', '3', '+', '6']
Output: ['1', '9']
```

For the example we are going to proceed with the following steps:

_Data processing_: 1-6

_Model processing_: 7-11

1. Generate Sum Pairs.
2. Integers to Padded Strings.
3. Integer Encoded Sequences.
4. One Hot Encoded Sequences.
5. Sequence Generation Pipeline.
6. Decode Sequences.
7. Problem definition.
8. Model compilation.
9. Fit the model.
10. Evaluate the model.
11. Make predictions.

## Further reading
- [Learning to Execute, 2015](http://arxiv.org/abs/1410.4615).
- [Sequence to Sequence Learning with Neural Networks, 2014](https://arxiv.org/abs/1409.3215).
- [Show and Tell: A Neural Image Caption Generator, 2014](https://arxiv.org/abs/1411.4555).
- [Caption this, with TensorFlow!](https://github.com/mlberkeley/oreilly-captions)
- [Keras example for addition with RNNs](https://github.com/keras-team/keras/blob/master/examples/addition_rnn.py)


## Homework
- Increase the number of terms or number of digits and tune the model to get 100% accuracy.
- Update the example to support a variable number of terms in a given instance and tune a model to get 100% accuracy.
- Add support for other math operations like subtraction, division, and multiplication.
- If you feel brave enough, take the "Caption this, with Tensorflow!" link. Can you make it work?

# 1. Generate Sum Pairs
We need a function to generate a sequence of random integers and their sum.

Look at the following code, do you understand what is going on?

In [ ]:
from random import seed
from random import randint

# generate lists of random integers and their sum
def random_sum_pairs(n_examples, n_numbers, largest):
    X, y = list(), list()
    for i in range(n_examples):
        in_pattern = [randint(1,largest) for _ in range(n_numbers)]
        out_pattern = sum(in_pattern)
        X.append(in_pattern)
        y.append(out_pattern)
    return X, y

In [ ]:
# Test the above function, play with the parameters a bit.
seed(1)
n_samples = 1
n_numbers = 2
largest = 10
# generate pairs
X, y = random_sum_pairs(n_samples, n_numbers, largest)
print(X, y)

# 2. Integers to Padded Strings
We need to convert the integers to strings. 
The important part here is that we need strings of the same size for the input and ouput. That is, we need to pad the data with a character, different from the data, so that the model learns to ingore it. We will use the space character ' ' for padding the string on the left, keeping the information far on the right.

**Homework**: Try different paddings and check the performance of the final model!

Padding requires we know the length of the biggest sequence. Since we write in base 10, we can take the ```log10()``` of the largest integer we can generate, times the number of summands ```n_numbers```, and add the right amount of plus symbols (```n_numbers - 1```). Taking care of the technicalities, we wet the following formula:

```python
max_length = n_numbers * ceil(log10(largest+1)) + n_numbers - 1
```
For example, if we assume we have numbers up to 10, with 3 summands, the longest sequence we can write is thus ```['1','0','+','1','0','+','1','0']``` for which the formula give us 
```python
max_length = 3 * ceil(log10(10+1)) + 3 - 1 = 3*2 + 3 - 1 = 8
```

Check the following code for converting lists of numbers to padded strings. Do you follow?

In [ ]:
from math import ceil
from math import log10

# convert data to strings
def to_string(X, y, n_numbers, largest):
	max_length = n_numbers * ceil(log10(largest+1)) + n_numbers - 1
	Xstr = list()
	for pattern in X:
		strp = '+'.join([str(n) for n in pattern])
		strp = ''.join([' ' for _ in range(max_length-len(strp))]) + strp
		Xstr.append(strp)
	max_length = ceil(log10(n_numbers * (largest+1)))
	ystr = list()
	for pattern in y:
		strp = str(pattern)
		strp = ''.join([' ' for _ in range(max_length-len(strp))]) + strp
		ystr.append(strp)
	return Xstr, ystr

In [ ]:
# Use the following code to test our functions so far. Feel free to play with the parameters.

seed(1)
n_samples = 1
n_numbers = 2
largest = 10
# generate pairs
X, y = random_sum_pairs(n_samples, n_numbers, largest)
print(X, y)
# convert to strings
X, y = to_string(X, y, n_numbers, largest)
print(X, y)

# 3. Integer Encoded Sequences

Neural networks are mathematical functions that take numbers as inputs. So we need to convert our strings to integers.
Integer encoding transforms the problem into a classification problem where the output sequence may be considered class outputs with 11 possible values each.

We can use the following alphabet:
```python
alphabet = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '+', ' ']
```

Look at the following code:

In [ ]:
# integer encode strings
def integer_encode(X, y, alphabet):
    char_to_int = dict((c, i) for i, c in enumerate(alphabet))
    Xenc = list()
    for pattern in X:
        integer_encoded = [char_to_int[char] for char in pattern]
        Xenc.append(integer_encoded)
    yenc = list()
    for pattern in y:
        integer_encoded = [char_to_int[char] for char in pattern]
        yenc.append(integer_encoded)
    return Xenc, yenc

In [ ]:
# We test our code so far

seed(1)
n_samples = 1
n_numbers = 2
largest = 10
# generate pairs
X, y = random_sum_pairs(n_samples, n_numbers, largest)
print(X, y)
# convert to strings
X, y = to_string(X, y, n_numbers, largest)
print(X, y)

# integer encode
alphabet = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '+', ' ']
X, y = integer_encode(X, y, alphabet)
print(X, y)

# 4. One Hot Encoded Sequences

[Wait what?](https://hackernoon.com/what-is-one-hot-encoding-why-and-when-do-you-have-to-use-it-e3c6186d008f) I did swallow the number argument in the previos step, but what is this one-hot-thingy and why do i need it? Take a look at the link and come back to the excercise. 

So, in short, we do not want the natural order of the categories to mess up our prediction problem. So a binary representation of the input will make sure there is no correlation between the categories, in our case, our digits.

This involves converting each integer to a binary vector with the same length as the alphabet and marking the specic integer with a 1. For example, a 0 integer represents the '0' character and would be encoded as a
binary vector with a 1 in the 0th position of an 11 element vector: [1, 0, 0, 0, 0, 0, 0, 0,0, 0, 0, 0].

In [ ]:
# one hot encode
def one_hot_encode(X, y, max_int):
    Xenc = list()
    for seq in X:
        pattern = list()
        for index in seq:
            vector = [0 for _ in range(max_int)]
            vector[index] = 1
            pattern.append(vector)
        Xenc.append(pattern)
    yenc = list()
    for seq in y:
        pattern = list()
        for index in seq:
            vector = [0 for _ in range(max_int)]
            vector[index] = 1
            pattern.append(vector)
        yenc.append(pattern)
    return Xenc, yenc

In [ ]:
# The following tests our code so far.

seed(1)
n_samples = 1
n_numbers = 2
largest = 10
# generate pairs
X, y = random_sum_pairs(n_samples, n_numbers, largest)
print(X, y)
# convert to strings
X, y = to_string(X, y, n_numbers, largest)
print(X, y)
# integer encode
alphabet = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '+', ' ']
X, y = integer_encode(X, y, alphabet)
print(X, y)
# one hot encode
X, y = one_hot_encode(X, y, len(alphabet))
print(X, y)

# 5. Sequence Generation Pipeline

We wrap everything we have so far in a singe function.

In [ ]:
from numpy import array

# generate an encoded dataset
def generate_data(n_samples, n_numbers, largest, alphabet):
    # generate pairs
    X, y = random_sum_pairs(n_samples, n_numbers, largest)
    # convert to strings
    X, y = to_string(X, y, n_numbers, largest)
    # integer encode
    X, y = integer_encode(X, y, alphabet)
    # one hot encode
    X, y = one_hot_encode(X, y, len(alphabet))
    # return as numpy arrays
    X, y = array(X), array(y)
    return X, y

# 6. Decode Sequences
Finally, we need to make sense of what we get out of our neural network, and decode back the output vectors into numbers. We first reverse the one-hot-encoding via the ```argmax``` function, and then use the reverse mapping from our alphabet. The following function takes care of that.

In [ ]:
from numpy import argmax

# invert encoding
def invert(seq, alphabet):
    int_to_char = dict((i, c) for i, c in enumerate(alphabet))
    strings = list()
    for pattern in seq:
        string = int_to_char[argmax(pattern)]
        strings.append(string)
    return ''.join(strings)

# 7. Problem definition
We need to define the specifications of the sequence prediction problem. We will use the following variables:
- **n_numbers**: The number of terms in the equation, (e.g. 2 for 10+10).
- **largest**: The largest numerical value for each term (e.g. 10 for values between 1-10).
- **alphabet**: The symbols used to encode the input and output sequences (e.g. 0-9, + and ' ')

We will use a configuration of the problem that has a modest complexity. Each instance will
be comprised of 3 terms with the maximum value of 10 per term. The alphabet remains fixed
regardless of configuration with the values 0-9, '+', and ' '.

The network needs three configuration values defined by the specification of the addition
problem:
- **n_chars**: The size of the alphabet for a single time step (e.g. 12 for 0-9, '+' and ' ').
- **n_in_seq_length**: The number of time steps of encoded input sequences (e.g. 8 for '10+10+10').
- **n_out_seq_length**: The number of time steps of an encoded output sequence (e.g. 2 for '30')

In [ ]:
# configure problem

# number of math terms
n_numbers = 3
# largest value for any single input digit
largest = 10
# scope of possible symbols for each input or output time step
alphabet = [str(x) for x in range(10)] + ['+', ' ']

# size of alphabet: (12 for 0-9, + and ' ')
n_chars = len(alphabet)
# length of encoded input sequence (8 for '10+10+10)
n_in_seq_length = n_numbers * ceil(log10(largest+1)) + n_numbers - 1
# length of encoded output sequence (2 for '30')
n_out_seq_length = ceil(log10(n_numbers * (largest+1)))

# 8. Model compilation

We are now ready to define the Encoder-Decoder LSTM. We will

- Use a single LSTM layer for the encoder and another single layer for the decoder. 
- Define the encoder with 75 memory cells.
- Define the decoder with 50 memory cells. 
- There is nothing special about the number of memory cells. It was found with a little trial and error. The asymmetry in layer sizes in the encoder and decoder seems like a natural organization given that input sequences are relatively longer than output sequences.
- Use the [categorical log loss](http://www.exegetic.biz/blog/2015/12/making-sense-logarithmic-loss/) for the 11 possible classes ('+' is not in the output sequece) that may be predicted in the output layer.
- Use the eficient Adam implementation of gradient descent and calculate accuracy during training and model evaluation.

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import TimeDistributed
from keras.layers import RepeatVector

# define LSTM
model = Sequential()
model.add(LSTM(75, input_shape=(n_in_seq_length, n_chars)))
model.add(RepeatVector(n_out_seq_length))
model.add(LSTM(50, return_sequences=True))
model.add(TimeDistributed(Dense(n_chars, activation='softmax')))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Yes, yes, I hear you, are we using over 52 K parameters to solve a simple addition problem!? But oh well, this is just to illustrate the technique.

# 9. Fit the model
The model is fit on a single epoch of 75,000 randomly generated instances of input-output pairs.
The number of sequences is a proxy for the number of training epochs. The total of 75,000
and a batch size of 32 were found with a little trial and error and are by no means an optimal
configuration. You are encouraged to play with this parameters to get a feeling about the fitting of the model.

Fitting the model provides a progress bar that shows the loss and accuracy of the model at
the end of each batch. The model does not take long to fit on the CPU. If the progress bar
interferes with your development environment, you can turn it off by setting verbose=0 in the
call to the fit() function.

In [ ]:
# fit LSTM
X, y = generate_data(75000, n_numbers, largest, alphabet)
model.fit(X, y, epochs=1, batch_size=32)

# 10. Evaluate the model
We can evaluate the model by generating predictions on 10000 different randomly generated
input-output pairs. The result will give an estimate of the model skill on randomly generated
examples in general.

Running the example prints both the log loss and accuracy of the model. Your specic
values may differ because of the stochastic nature of neural networks, but the model accuracy
should be in the high 90s.

In [ ]:
# evaluate LSTM
X, y = generate_data(10000, n_numbers, largest, alphabet)
loss, acc = model.evaluate(X, y, verbose=0)
print('Loss: %f, Accuracy: %f' % (loss, acc*100))

# 11. Make Predictions with the Model

We can make predictions using the fit model. We will demonstrate making one prediction at
a time and provide a summary of the decoded input, expected output, and predicted output.
Printing the decoded output gives us a more concrete connection to the problem and model
performance. Here, we generate 10 new random input-output sequence pairs, make a prediction
using the fit model for each, decode all the sequences involved, and print them to the screen.

In [ ]:
# fancy printing in colors
class colors:
    ok = '\033[92m'
    fail = '\033[91m'
    close = '\033[0m'
    
# predict
for _ in range(10):
    # generate an input-output pair
    X, y = generate_data(1, n_numbers, largest, alphabet)
    # make prediction
    yhat = model.predict(X, verbose=0)
    # decode input, expected and predicted
    out_seq = invert(y[0], alphabet)
    predicted = invert(yhat[0], alphabet)
    if out_seq == predicted:   
        print(colors.ok + '☑' + colors.close, end=' ')
    else:
        print(colors.fail + '☒' + colors.close, end=' ')
    print('%s = %s (when expecting %s)' % (in_seq, predicted, out_seq))        

Run the code several times in the cell above and look at the predictions. Does the model get everything right all the time?

## Save your model

You can save the model architecture (layers and how they connect) and weights (arrays
of numbers) to separate files. This is the recommended approach as it allows you to update the
model weights at a later stage while ensuring the model architecture is left unchanged. Maybe you want to update your model weights without changing the architecture?

Keras provides two formats for preserving the model architecture: JSON and YAML formats.
The difference being in having a human readable format or not. 
The following code sample saves the architecture in a JSON file. 
The weights are saved in a [HDF5](https://en.wikipedia.org/wiki/Hierarchical_Data_Format) file format.

In [ ]:
# convert model architecture to JSON format
architecture = model.to_json()
# save architecture to JSON file
with open('architecture.json', 'wt') as json_file:
    json_file.write(architecture)
# save weights to hdf5 file
model.save_weights('weights.h5')

In [ ]:
from keras.models import model_from_json

# load architecture from JSON File
json_file = open('architecture.json', 'rt')
architecture = json_file.read()
json_file.close()
# create model from architecture
loaded_model = model_from_json(architecture)
# load weights from hdf5 file
loaded_model.load_weights('weights.h5')

In [ ]:
class colors:
    ok = '\033[92m'
    fail = '\033[91m'
    close = '\033[0m'

# make predictions again
for _ in range(10):
    # generate an input-output pair
    X, y = generate_data(1, n_numbers, largest, alphabet)
    # make prediction
    yhat = loaded_model.predict(X, verbose=0)
    # decode input, expected and predicted
    in_seq = invert(X[0], alphabet)
    out_seq = invert(y[0], alphabet)
    predicted = invert(yhat[0], alphabet)
    if out_seq == predicted:   
        print(colors.ok + '☑' + colors.close, end=' ')
    else:
        print(colors.fail + '☒' + colors.close, end=' ')
    print('%s = %s (when expecting %s)' % (in_seq, predicted, out_seq))        

# What to do next?

Congratulations! You have come so far. Now take a look at the homework and play a bit more!